In [2]:
'''
run segmentation algo on images
compare to ground truth
pixel by pixel. note, labels have a 'void' barrier (label 255) around objects that doesn't count for anything
'''

"\nrun segmentation algo on images\ncompare to ground truth\npixel by pixel. note, labels have a 255 'void' barrier around objects that doesn't count\n"

In [26]:
# https://pytorch.org/hub/pytorch_vision_deeplabv3_resnet101/

import torch

# use deeplabv3_resnet50 instead of deeplabv3_resnet101 to reduce the model size
model = torch.hub.load('pytorch/vision:v0.8.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

scriptedm = torch.jit.script(model)
torch.jit.save(scriptedm, "deeplabv3_scripted.pt")

Using cache found in /home/tim/.cache/torch/hub/pytorch_vision_v0.8.0


In [27]:
filename = "2007_000032.jpg"
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
input_image = Image.open(filename)
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)['out'][0]
output_predictions = output.argmax(0)

In [28]:
""" 
https://gist.github.com/wllhf/a4533e0adebe57e3ed06d4b50c8419ae
Python implementation of the color map function for the PASCAL VOC data set. 
Official Matlab version can be found in the PASCAL VOC devkit 
http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit
"""
import numpy as np
from skimage.io import imshow
import matplotlib.pyplot as plt

def color_map(N=256, normalized=False):
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3

        cmap[i] = np.array([r, g, b])

    cmap = cmap/255 if normalized else cmap
    return cmap

cmap = color_map()

In [29]:
# # create a color pallette, selecting a color for each class
# palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
# colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
# colors = (colors % 255).numpy().astype("uint8")

# # plot the semantic segmentation predictions of 21 classes in each color
# r = Image.fromarray(output_predictions.byte().cpu().numpy()).resize(input_image.size)
# r.putpalette(color_map())

# import matplotlib.pyplot as plt
# plt.imshow(r)

In [30]:
target = Image.open("2007_000032.png")
target = np.array(target)
target = torch.tensor(target)

In [31]:
void_pixels = torch.count_nonzero(target==255)

In [32]:
target_voids = target == 255
output_predictions[target_voids] = 255
accuracy = (torch.count_nonzero(output_predictions.cpu()==target) - void_pixels) / (target.shape[0] * target.shape[1])


In [33]:
print(accuracy)

tensor(0.9460)
